In [1346]:
import pandas as pd
import numpy as np

In [1347]:
pl0 = pd.read_csv("Kraftwerksliste_CSV.csv", error_bad_lines=False, engine="python", encoding="latin1", sep=";", skiprows=9)
bm = pd.read_csv("block_plant_mapper.csv", error_bad_lines=False, engine="python", sep=";")

In [1348]:
pl0.rename(columns={ pl0.columns[10]: "Energieträger_old", pl0.columns[13]: "Energieträger"}, inplace=True)

In [1349]:
#bm
pl1 = pl0.loc[np.logical_or(pl0.Energieträger == "Kernenergie", np.logical_or(pl0.Energieträger == "Erdgas", np.logical_or(pl0.Energieträger == "Steinkohle", pl0.Energieträger == "Braunkohle"))), :]

In [1350]:
# pl1

In [1351]:
pl2 = pl1.rename(columns={ pl1.columns[0]: "BlockID", pl1.columns[3]: "PLZ", pl1.columns[4]: "Ort", pl1.columns[8]: "Inbetriebnahme", pl1.columns[9]: "Status", pl1.columns[15]: "KWK", pl1.columns[16]: "Nettoleistung"})
#pl3 = pl2.rename(columns={ pl1.columns[1]: "BlockID" })

In [1352]:
pl2['Nettoleistung'] = pl2['Nettoleistung'].str.replace('.','')
pl2['Nettoleistung'] = pl2['Nettoleistung'].str.replace(',','.')
#pl2['Nettoleistung'] = pl2['Nettoleistung'].str.replace('\n','')
#pl2['Nettoleistung'] = pl2[pd.to_numeric(pl2["Nettoleistung"])]
#pl2['Nettoleistung'] = pl2['Nettoleistung'].str.replace(';','.')

In [1353]:
#pl2[] = pl2[pd.to_numeric(pl2["Nettoleistung"])]
pl2['Nettoleistung'] = pl2['Nettoleistung'].apply(pd.to_numeric)

In [1354]:
# pl2

In [1355]:
cols = [10, 11,12,14,17,18,19]
pl3 = pl2.drop(pl2.columns[cols],axis=1)

In [1356]:
# pl3

In [1357]:
# Status: endgültig stillgelegt, vorläufig stillgelegt, in Betrieb, Sicherheitsbereitschaft, Sonderfall, 
# Stillegung: [Jahreszahl]
# StA: ja/ nein

In [1358]:
newdf = pd.DataFrame()

In [1359]:
def generate_decom_tuple(state):
    if state == "In Betrieb":
        return("in Betrieb", "0")
    elif "Endgültig Stillgelegt" in state:
        a = "stillgelegt"
        b = str(int(state[22:26]))
        return (a, b)
    elif "Vorläufig Stillgelegt" in state:
        a = "vorläufig stillgelegt"
        return (a, "0")
    else:
        return(state, "0")

In [1360]:
def fix_company(company):
    cmp = company
    company_prefix, _ = company.split(" " , 1)
    # company_dict = {}
    company_dict = {"RWE": "RWE AG", "Vattenfall": "Vattenfall GmbH", "Uniper": "Uniper SE", "EnBW": "EnBW AG", "Steag": "Steag GmbH"}
    try:
        cmp = company_dict[company_prefix]
    except KeyError:
        return company
    return cmp

In [1361]:
for index, row in pl3.iterrows():
    #if row['Status'] == "Endgültig Stillgelegt 2016 (mit StA)":
    # print(row)
    if row['KWK'] == "nein":
        row['KWK'] = "Nein" # workaround for BoA plants
    status, stilllegung = generate_decom_tuple(row['Status'])
    cmp = fix_company(row['Unternehmen'])
    row['Unternehmen'] = cmp
    row['Status'] = status
    row['Stilllegung'] = stilllegung
    newdf = newdf.append(row)
    

In [1362]:
# newdf['Stilllegung'] = newdf['Stilllegung'].astype(int)
#newdf['Stilllegung'] = newdf['Stilllegung'].apply(lambda x: int(x) if x == x else None)

In [1363]:
newdf['Stilllegung'] = newdf['Stilllegung'].replace("0", np.nan)

In [1364]:
pl3 = newdf

In [1365]:
atest = newdf.groupby(["Unternehmen"]).count()
atest.sort_values("BlockID", inplace=True, ascending=False)

In [1366]:
# pl3

In [1367]:
atest = pl3.groupby(["Status"]).count()
# atest

In [1368]:
# pl3.rename(columns={ pl3.columns[10]: "Energieträger"}, inplace=True)

In [1369]:
pl4 = bm.merge(pl3, how="outer", on="BlockID")

In [1370]:
pl5 = pl4[pd.notnull(pl4['BlockID'])]
#pl5 = pl4
pl6 = pl5[pd.notnull(pl5['Nettoleistung'])]

In [1371]:
# pl6

In [1372]:
blocks = pl6.copy()
stammdaten = pl6.copy()

In [1373]:
blocks.sort_values("Nettoleistung", inplace=True, ascending=False)
# blocks

In [1374]:
plants = blocks.loc[:]
plants = plants.dropna(subset=["KraftwerkID"])

In [1375]:
plants_d = plants.loc[plants.duplicated("KraftwerkID", keep=False)]
# plants_d

In [1376]:
plants_c = plants_d.groupby(["KraftwerkID"]).sum()
plants_count = plants_d.groupby(["KraftwerkID"]).count()

In [1377]:
plants_f = plants_c.reset_index()

In [1378]:
plants_count = plants_count.loc[:, ["BlockID"]]

In [1379]:
plants_count = plants_count.reset_index()
# plants_count 

In [1380]:
plants_f = plants_f.merge(plants_count, on="KraftwerkID", how="inner")
# plants_f

In [1381]:
plants_a = plants.drop_duplicates(subset="KraftwerkID")

In [1382]:
plants_b = plants_d.loc[:]

In [1383]:
plants_f = plants_f.rename(columns={ "Nettoleistung": "Gesamtleistung", "BlockID": "Blockzahl"})

In [1384]:
plants_f = plants_f.merge(plants_a, on="KraftwerkID", how="left")
# plants_f

In [1385]:
blocks.drop_duplicates(subset="BlockID", inplace=True)

In [1386]:
#cols = [4,5,6,8]
#blocks.drop(blocks.columns[cols],axis=1, inplace=True)
#test_blocks = blocks.drop(blocks.columns[cols],axis=1)

In [1387]:
blocks.drop(['Ort', 'PLZ', 'Straße und Hausnummer (Standort Kraftwerk)', 'Blockname'], axis=1, inplace=True)

In [1400]:
# blocks

In [1389]:
# stammdaten

In [1390]:
# test_blocks

In [1391]:
#blocks.loc[blocks.Energieträger == 'Erdgas',]

In [1392]:
plants_2 = plants.rename(columns={plants.columns[6]: "Anschrift"})
# plants_2

In [1393]:
# plants_2

In [1394]:
cols = [2, 3, 8, 9, 10, 11, 12]
plants_3 = plants_2.drop(plants_2.columns[cols],axis=1, )
# plants_3

In [1395]:
# stammdaten

In [1396]:
# cols = [0,2,3,9,10,11,12]
drop_list = ['KraftwerkID', 'Blockname', 'Energieträger', 'Inbetriebnahme', 'KWK', 'Kraftwerksname', 'Nettoleistung', 'Status', 'Stilllegung', 'Unternehmen']
stammdaten.drop(drop_list, axis=1, inplace=True)

In [1397]:
stammdaten.to_csv("stammdaten.csv", index=False)

In [1398]:
blocks.to_csv("blocks_3.csv", index=False)

In [1399]:
stammdaten.to_csv("stammdaten_nh.csv", index=False, header=False)
blocks.to_csv("blocks_nh.csv", index=False, header=False)